In [4]:
import pickle
import pandas as pd
import numpy as np
import re

### Load data

In [5]:
entdicf = "entdic3hops.pkl"
reldicf = "reldic3hops.pkl"
triplesf = "triples3hops.pd.pkl"
allrelsf = "allrel3hops.pd.pkl"
typesf = "../dbpedia/dbpcore.nt.types.dok"
bigentdicf = "../dbpedia/dbpcore.nt.entdic.ssd"

In [6]:
entdic = pickle.load(open(entdicf))
reldic = pickle.load(open(reldicf))
triples = pickle.load(open(triplesf))
allrels = pickle.load(open(allrelsf))
reventdic = {v: k for k, v in entdic.items()}

In [22]:
# load dictionaries
def loadDic(entdicf):
    entdic = {}
    for line in open(entdicf):
        splits = line.split(" ")
        entdic[splits[0]] = int(splits[1])
    return entdic

# load big dictionaries
bigentdic = loadDic(bigentdicf)
print "big entity dict loaded"
'''bigreldic = loadDic(bigreldicf)
print "big relation dict loaded"'''

big entity dict loaded


'bigreldic = loadDic(bigreldicf)\nprint "big relation dict loaded"'

### Direct compatibility data

In [31]:
direct_forward = allrels[allrels.d == "F"][["e","r"]].drop_duplicates()
direct_reverse = allrels[allrels.d == "R"][["e","r"]].drop_duplicates()

In [32]:
# save
outdf = open("direct_forward.ssd", "w")
for _, row in direct_forward.iterrows():
    outdf.write("%d %d\n" % (row["e"], row["r"]))
outdf.close()
outdf = open("direct_reverse.ssd", "w")
for _, row in direct_reverse.iterrows():
    outdf.write("%d %d\n" % (row["e"], row["r"]))
outdf.close()

### Type data

In [20]:
allents = entdic.keys()

In [23]:
allentbidxs = set(map(lambda x: bigentdic[x], allents))

In [24]:
# go through the types file
enttypes = {}
c = 0
typesused = set()
for line in open(typesf):
    c += 1
    if c % 1000000 == 0:
        print c
    ns = [int(x) for x in line.split(" ")]
    if len(ns) == 2 and ns[0] in allentbidxs:
        if ns[0] not in enttypes:
            enttypes[ns[0]] = set()
        enttypes[ns[0]].add(ns[1])
        typesused.add(ns[1])
        

1000000
2000000
3000000
4000000
5000000
6000000
7000000


In [25]:
revbigentdic = {v: k for k, v in bigentdic.items()}

In [26]:
ntypesused = map(lambda x: revbigentdic[x], typesused)
ntypesused = sorted(ntypesused)
ntypesidx = range(max(reldic.values())+1, max(reldic.values())+1+len(ntypesused))
ntypesdic = dict(zip(ntypesused, ntypesidx))

reindex types

In [27]:
def maketypelist(enttypes):
    for et in enttypes.items():
        for v in et[1]:
            yield (et[0], v)

nenttypes = [i for i in maketypelist(enttypes)]
enttypesdf = pd.DataFrame(nenttypes)
enttypesdf.columns = ["e", "t"]
enttypesdf["e"] = enttypesdf["e"].map(lambda x: entdic[revbigentdic[x]])
enttypesdf["t"] = enttypesdf["t"].map(lambda x: ntypesdic[revbigentdic[x]])

In [28]:
# save
outtypesf = open("types.ssd","w")
for _, row in enttypesdf.iterrows():
    outtypesf.write("%d %d\n" % (row["e"], row["t"]))

## Trans compatibility data

### Trans forward

In [176]:
enttypesdf[enttypesdf.e == 9105]

,e,t
13735,9105,535


In [180]:
def relc(direct, types=None):
    def mapper(eids):
        acc = []
        for eid in eids:
            ret = []
            # relations using the direct dataframe
            rels = set(direct.ix[direct.e == eid, "r"].values.flat)
            ret += list(rels)
            # types
            if types is not None:
                typs = set(types.ix[types.e == eid, "t"].values.flat)
                ret += list(typs)
            acc += ret
        rdic = {}
        for rete in acc:
            if rete not in rdic:
                rdic[rete] = 0
            rdic[rete] += 1
        for k in rdic:
            rdic[k] *= 1./len(eids)
        return rdic
    return mapper
        

In [181]:
tf = triples.groupby(["s", "r"]).agg(lambda x: set(x.values)).reset_index() # triples has small indexes

In [182]:
tff = pd.DataFrame(columns=["s", "r", "o"])
tff["s"] = tf["s"]
tff["r"] = tf["r"]
tff["o"] = tf["o"].map(relc(direct_forward, enttypesdf))

In [183]:
tffof = open("trans_fwd_fwd.ssd","w")
for _, row in tff.iterrows():
    s = row["s"]
    r = row["r"]
    for oe in row["o"]:
        tffof.write("%d %d %d %f\n" % (s, r, oe, row["o"][oe]))

In [184]:
tfr = pd.DataFrame(columns=["s", "r", "o"])
tfr["s"] = tf["s"]
tfr["r"] = tf["r"]
tfr["o"] = tf["o"].map(relc(direct_reverse))

In [185]:
tfrof = open("trans_fwd_rev.ssd","w")
for _, row in tfr.iterrows():
    s = row["s"]
    r = row["r"]
    for oe in row["o"]:
        tfrof.write("%d %d %d %f\n" % (s, r, oe, row["o"][oe]))

In [186]:
tr = triples.groupby(["o", "r"]).agg(lambda x: set(x.values)).reset_index()

In [187]:
# forward rel and type counts
trf = pd.DataFrame(columns=["o","r","s"])
trf["o"] = tr["o"]
trf["r"] = tr["r"]
trf["s"] = tr["s"].map(relc(direct_forward, enttypesdf))

In [188]:
# save trans reverse forward
trfof = open("trans_rev_fwd.ssd", "w")
for _, row in trf.iterrows():
    o = row["o"]
    r = row["r"]
    for se in row["s"]:
        trfof.write("%d %d %d %f\n" % (o, r, se, row["s"][se]))

In [189]:
trr = pd.DataFrame(columns=["s","r","o"])
trr["o"] = tr["o"]
trr["r"] = tr["r"]
trr["s"] = tr["s"].map(relc(direct_reverse))

In [190]:
# save trans reverse reverse
trrof = open("trans_rev_rev.ssd", "w")
for _, row in trr.iterrows():
    o = row["o"]
    r = row["r"]
    for se in row["s"]:
        trrof.write("%d %d %d %f\n" % (o, r, se, row["s"][se]))

## Playing with data

In [191]:
for entdice in entdic:
    if "Avenger" in entdice:
        print entdice
        print entdic[entdice]

http://dbpedia.org/resource/Captain_America:_The_First_Avenger
8329
http://dbpedia.org/resource/Avengers:_Age_of_Ultron
9105
http://dbpedia.org/resource/The_Avengers_(2012_film)
10503


In [220]:
tfr.ix[tfr.s == 9105].iloc[0,2]

{3: 0.07142857142857142,
 14: 1.0,
 15: 0.9285714285714285,
 21: 0.07142857142857142,
 26: 0.42857142857142855,
 27: 0.07142857142857142,
 28: 0.21428571428571427,
 30: 0.21428571428571427,
 31: 0.21428571428571427,
 33: 0.07142857142857142,
 38: 0.14285714285714285,
 54: 0.07142857142857142,
 73: 0.9285714285714285,
 81: 0.5714285714285714,
 117: 0.21428571428571427,
 130: 0.42857142857142855,
 131: 0.42857142857142855,
 141: 0.14285714285714285,
 168: 0.07142857142857142,
 169: 0.07142857142857142,
 181: 0.14285714285714285,
 183: 0.07142857142857142,
 186: 0.07142857142857142,
 187: 0.07142857142857142,
 193: 0.14285714285714285,
 198: 0.07142857142857142,
 204: 0.21428571428571427,
 208: 0.07142857142857142,
 210: 0.07142857142857142,
 216: 0.5714285714285714,
 221: 0.07142857142857142,
 222: 0.21428571428571427,
 234: 0.21428571428571427,
 239: 0.07142857142857142,
 348: 0.07142857142857142,
 400: 0.07142857142857142,
 409: 0.07142857142857142,
 425: 0.07142857142857142}

In [106]:
revtypdic = {v: k for k, v in ntypesdic.items()}

In [219]:
revtypdic[559]

'http://dbpedia.org/ontology/Person'

In [7]:
reventdic = {v: k for k, v in entdic.items()}
revreldic = {v: k for k, v in reldic.items()}

In [197]:
reventdic[1]

'http://dbpedia.org/resource/Larry_Peerce'

In [206]:
revreldic[500]

'http://dbpedia.org/property/julRecordLowF'

In [227]:
revreldic[81]

'http://dbpedia.org/property/aux'

In [208]:
tf[tf.s==9105].iloc[0,2]

{1171,
 1341,
 1598,
 2657,
 2844,
 3413,
 5163,
 7502,
 9896,
 10394,
 10437,
 10553,
 10557,
 11246}

In [214]:
reventdic[2657]

'http://dbpedia.org/resource/Scarlett_Johansson'

In [215]:
enttypesdf[enttypesdf.e == 2657]

,e,t
14442,2657,625
14443,2657,630
14444,2657,600
14445,2657,613
14446,2657,603
14447,2657,504
14448,2657,602
14449,2657,559
14450,2657,605


In [32]:
ntypesdic["http://dbpedia.org/ontology/Film"]

535

In [33]:
reldic["http://dbpedia.org/ontology/director"]

31